In [16]:
#%pip install --upgrade tensorflow

                                              0.0/276.5 MB ? eta -:--:--
                                              0.1/276.5 MB 3.2 MB/s eta 0:01:27
                                              0.3/276.5 MB 5.2 MB/s eta 0:00:54
                                             1.2/276.5 MB 11.1 MB/s eta 0:00:25
                                             2.5/276.5 MB 16.1 MB/s eta 0:00:18
                                             2.9/276.5 MB 18.7 MB/s eta 0:00:15
                                             5.4/276.5 MB 23.1 MB/s eta 0:00:12
     -                                       8.1/276.5 MB 30.3 MB/s eta 0:00:09
     -                                      10.5/276.5 MB 40.9 MB/s eta 0:00:07
     -                                      13.0/276.5 MB 54.7 MB/s eta 0:00:05
     --                                     15.9/276.5 MB 65.6 MB/s eta 0:00:04
     --                                     18.3/276.5 MB 65.6 MB/s eta 0:00:04
     --                                     20.

ERROR: Could not install packages due to an OSError: [WinError 5] Accesso negato: 'C:\\Users\\USER\\anaconda3\\Lib\\site-packages\\~ensorflow\\compiler\\tf2tensorrt\\_pywrap_py_utils.pyd'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import tensorflow as tf
import os

# Definizione dei parametri di caricamento delle immagini
batch_size = 32
image_size = (816, 612)
directory = '../Dataset/JPEGImages/T'
test_split = 0.2

image_files = os.listdir(directory)
# Caricamento delle immagini dalla directory
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory,
    class_names=None,
    batch_size=batch_size,
    validation_split=test_split,
    subset="training",
    seed=123
)

class_names = os.listdir(directory)
print("Nomi delle classi:", class_names)

# Calcolo del numero di campioni per il set di test
num_samples = train_dataset.cardinality().numpy()
num_test_samples = int(num_samples * test_split)

# Divisione del set di addestramento in set di addestramento e test
test_dataset = train_dataset.take(num_test_samples)
train_dataset = train_dataset.skip(num_test_samples)

Found 1477 files belonging to 20 classes.
Using 1182 files for training.
Nomi delle classi: ['Apple', 'Banana', 'Bread', 'Bun', 'Doughnut', 'Egg', 'Fired_dough', 'Grape', 'Lemon', 'Litchi', 'Mango', 'Mix', 'Mooncake', 'Orange', 'Peach', 'Pear', 'Plum', 'Qiwi', 'Sachima', 'Tomato']


In [ ]:
%pip install tensorflow
%pip install tf_slim
%pip install tensorflow-hub
%pip install tensorflow-object-detection-api
%pip install tf-models-official
%pip install opencv-python


In [26]:
import tensorflow as tf
import cv2
import numpy as np
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

# Percorso al modello pre-addestrato
PATH_TO_SAVED_MODEL = '../Preprocessed/faster_rcnn_resnet152_v1_1024x1024_coco17_tpu-8.config'  # Sostituisci con il percorso del modello pre-addestrato

# Percorso alla mappa delle etichette
PATH_TO_LABELS = '../Dataset/JPEGImages/T'  # Sostituisci con il percorso della mappa delle etichette

# Carica il modello pre-addestrato
model = tf.saved_model.load()

# Carica la mappa delle etichette
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

# Effettua il rilevamento oggetti
detections = model(PATH_TO_LABELS)

# Visualizza i risultati del rilevamento
viz_utils.visualize_boxes_and_labels_on_image_array(
    image_np,
    detections['detection_boxes'][0].numpy(),
    (detections['detection_classes'][0].numpy() + 1).astype(int),
    detections['detection_scores'][0].numpy(),
    category_index,
    use_normalized_coordinates=True,
    max_boxes_to_draw=200,
    min_score_thresh=0.3,
    agnostic_mode=False
)

# Mostra l'immagine con le detection
image_np = cv2.imread(image_path)
image_np = cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB)
cv2.imshow('Object Detection', image_np)
cv2.waitKey(0)
cv2.destroyAllWindows()


UnimplementedError: File system scheme 'https' not implemented (file: 'https://tfhub.dev/tensorflow/faster_rcnn/resnet152_v1_1024x1024/inference/1/saved_model.pb')

In [17]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Dimensioni delle immagini per ResNet50 (224x224)
input_shape = (224, 224, 3)

# Numero di classi del tuo problema di classificazione
num_classes = 20  # Assumi che ci siano 10 classi, sostituisci con il tuo numero reale

# Preprocessamento delle immagini
preprocess_input = tf.keras.applications.resnet50.preprocess_input

# Creazione del modello R-CNN
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=input_shape))
base_model.trainable = False

input_layer = Input(shape=input_shape)
preprocessed_input = preprocess_input(input_layer)

rcnn_features = base_model(preprocessed_input)
flatten_features = tf.keras.layers.Flatten()(rcnn_features)
output_layer = tf.keras.layers.Dense(num_classes, activation='softmax')(flatten_features)

model = Model(inputs=input_layer, outputs=output_layer)

# Compilazione del modello
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Utilizza ImageDataGenerator per caricare le immagini dal percorso specificato
datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)
train_dataset = datagen.flow_from_directory(
    directory,
    target_size=input_shape[:2],  
    batch_size=64,           
    class_mode='sparse'
)

# Callback di Early Stopping
early_stopping = EarlyStopping(patience=3, restore_best_weights=False)

# Addestramento del modello con Early Stopping
model.fit(train_dataset, epochs=10, batch_size=64,  callbacks=[early_stopping])

# Addestramento del modello
#model.fit(train_dataset,epochs=5, batch_size=64)


Found 1477 images belonging to 20 classes.
Epoch 1/10
24/24 [==============================] - 131s 5s/step - loss: 11.3596 - accuracy: 0.6249
Epoch 2/10
18/24 [=====================>........] - ETA: 34s - loss: 1.4946 - accuracy: 0.9201

KeyboardInterrupt: 

In [ ]:
predictions = model.predict(test_dataset)
y_true = test_dataset.classes
y_pred = tf.argmax(predictions, axis=1)

# Calcolo delle metriche
accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
precision = precision_score(y_true, y_pred, average='macro')

# Stampa delle metriche
print("Accuracy:", accuracy)
print("F1 Score:", f1)
print("Recall:", recall)
print("Precision:", precision)